<a href="https://colab.research.google.com/github/SahajSohi/Aircraft-Routing-Optimization-project/blob/main/DP_Aircraft_Routing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
import random
import copy
import time
import plotly.express as px
import matplotlib.pyplot as plt
random.seed(0)
np.random.seed(0)

Reading the Input Data File

In [ ]:
df=pd.read_excel('/content/Flight_Schedule (Final Version).xlsx')

Creating the Classes

In [ ]:
class airport:

    def __init__(self, aport, aircraftCount_dict):
        self.aport = aport
        self.aircraftCount_dict = aircraftCount_dict

class originAirport:

    def __init__(self, airport, deptime, depday):
        self.airport = airport
        self.deptime = deptime
        self.depday = depday
class destinationAirport:

    def __init__(self, airport, arrtime, arrday, elapsed_time, total_prop_delay, flight_time):
        self.airport = airport
        self.arrtime = arrtime
        self.arrday = arrday
        self.elapsed_time = elapsed_time
        self.total_prop_delay = total_prop_delay
        self.flight_time = flight_time

class FlightLeg:

    def __init__(self,origin,dest,flights,day,arr_delay,dep_datetime,arr_datetime,flighttime,
                 total_propagated_delay,pointer,elapsed_time, processed, firstnodeorigin, fleet):
        self.origin = origin
        self.dest = dest
        self.flights = flights
        self.day = day
        self.arr_delay = arr_delay
        self.dep_datetime = self.dateTimeObject(str(dep_datetime))
        self.arr_datetime = self.dateTimeObject(str(arr_datetime))
        self.flighttime = flighttime
        self.total_propagated_delay = total_propagated_delay
        self.pointer = pointer
        self.elapsed_time = elapsed_time
        self.processed = processed
        self.firstnodeorigin = firstnodeorigin
        self.fleet = fleet

    def dateTimeObject(self,String):
        return datetime.datetime.strptime(String,"%Y-%m-%d %H:%M:%S")

Calculating Slack Time, Propogated Delays and Elapsed Time

In [ ]:
def calculate_slack_time_seconds(Dept,Arr, day1, day2, mean_turnaround_time):
    if day1 >= day2:
        return ((Dept-Arr).total_seconds() - mean_turnaround_time)
    else:
        return -99

def check_slack_time_seconds_validity(slack_time):
    return (24.*3600)>=slack_time>=0 #Check for flights within stipulated time for connection not beyond that

def check_time_elapsed_validity_seconds(time_elapsed):
    return (0<=time_elapsed<=36*60*60)

def calculate_propagated_delay_seconds(prev_prop_delay,arrival_delay,slack_time):
    prop_delay = prev_prop_delay+arrival_delay-slack_time # when arrival delay is in seconds
    return max(0, prop_delay)

def calculate_time_elapsed(prev_elapsed_time,Arr,Dept):
    return prev_elapsed_time+(Dept-Arr).total_seconds()

Optimization Model

In [ ]:
#Optimization master problem
#def route_optimizer(stagewise_routes , prop_delay , route_strings , sflights , assignedFlights , itr, penaltyCost , delayWeight):

    model = ConcreteModel()
    n = len(stagewise_routes)

    uniqueFlights = []
    multipleFlight = []
    #fp_dict = {}

    for sr in stagewise_routes:
        for s in sr:
            uniqueFlights.append(s)

    uniqueFlights = set(uniqueFlights)

   #m = len(sflights)
   #nparray = []
   #flightString = {}
    #ff = 0
    for i in sflights:
        if ff not in flightString:
            flightString[ff] = []

        #npList=[]
        #cc = 0
        for j in stagewise_routes:
            if i in j:
                npList.append(1)
                flightString[ff].append(cc)
            else:
                npList.append(0)
            cc+=1
        nparray.append(npList)
        ff+=1
    varList = []
    fvarList = []
    stringFlight = {}
    flightString = {}

    for j in range(len(sflights)):
        fvarList.append(j)
        if j not in flightString:
            flightString[j] = []

    for i in range(n):
        if i not in stringFlight:
            stringFlight[i] = []

            for s in sflights:
                if s in stagewise_routes[i]:
                    stringFlight[i].append(sflights.index(s))
                    flightString[sflights.index(s)].append(i)

    for i in range(n):
        varList.append(i)

    for s in sflights:
        mf = 0
        for key, value in stringFlight.items():
            if sflights.index(s) in value:
                mf+=1
        if mf > 1:
            multipleFlight.append(sflights.index(s))

    model.xVar = Var(varList, within = Integers, bounds = (0,1), initialize = 0)
    model.fVar = Var(fvarList, bounds = (0,1), initialize = 0)

    ft = 0
    if len(assignedFlights) > 0:
        model.assignedFlights_constraint = ConstraintList()
        for f in sflights:
            if f in assignedFlights:
                model.assignedFlights_constraint.add(model.fVar[ft] == 1)
            ft+=1

    model.numberFlights_constraint = Constraint(expr = sum(model.fVar[sflights.index(f)] for f in sflights
                                                           if f not in assignedFlights) <= priority_number)

    model.cover_constraint = ConstraintList()
    for i in range(m):
        a=nparray[i]
        add=0

        for j in range(n):
            k=a[j]*model.xVar[j]
            add=add+k

        model.cover_constraint.add(add <=1)

    model.airport_capacity_constraint = ConstraintList()
    model.airport_capacity_constraint.add(sum(model.xVar[i] for i in range(len(model.xVar))) <= len(model.xVar))

    model.route_flight_constraint = ConstraintList()
    for i in range(n):
        model.route_flight_constraint.add(model.xVar[i] >= sum(model.fVar[j] for j in stringFlight[i]
                                                               if j not in multipleFlight)/len(stringFlight[i]))

    model.multiple_flight_constraint = ConstraintList()
    for key, value in flightString.items():
        if key in multipleFlight:
            model.multiple_flight_constraint.add(model.fVar[key] <= sum(model.xVar[v] for v in value))

    model.flight_string_constraint = ConstraintList()
    for item, value in flightString.items():
        model.flight_string_constraint.add(sum(model.xVar[v] for v in value) >= model.fVar[item])

    aircraftRouting = []
    priority = []
    penaltyProd = []
    reward = []

    priority.append(sum(model.fVar[j] * uniquePriorities[j] for j in range(len(sflights))))
    for i in range(n):
        penaltyProd.append(penaltyCost * len(stagewise_routes[i]) * model.xVar[i])
        aircraftRouting.append(prop_delay[i] * model.xVar[i])

    model.value = Objective(expr = sum(aircraftRouting) * delayWeight - sum(priority) * priorityWeight
                            - sum(penaltyProd))


    result_obj = SolverFactory('mindtpy').solve(model, mip_solver='glpk', nlp_solver = 'ipopt', tee = True)

    sol_routes = []
    sol_delay = []
    index = []
    flt = []
    stringIndex = []
    c = 0
    for v in range(n):
        if model.xVar[v].value != 0:
            sol_routes.append(model.xVar[v].value)
            sol_delay.append(prop_delay[v])
            index.append(c)
            stringIndex.append(v)
        c+=1

    for f in range(len(sflights)):
        if model.fVar[f].value > 0.9:
            flt.append(sflights[f])


    return sol_routes, index, flt, sol_delay

In [ ]:
origin_dict_og = {}
destination_dict_og = {}
object_dict_og = {}
object_dict_copy = {}
object_dict_dep_sort = {}
fleet_cap_dict = {}
airport_aircraft_dict = {}
airport_cluster_dict = {}
cluster_aircraft_dict = {}
aircraftStringCount = {}
operatingCost_dict = {}
equipment = []
allAirports = []
allAircrafts = []
ogRevenue = []
flightCount = {}

#Sort the flight leg nodes as per the time.

for i,r in df.iterrows():
    allAirports.append(r["ORIGIN"])
    allAirports.append(r["DEST"])
#     allAircrafts.append(r["TAIL_NUM"][-2:])
    if r["FLIGHTS"] not in flightCount:
        flightCount[r["FLIGHTS"]] = 1
    elif r["FLIGHTS"] in flightCount:
        flightCount[r["FLIGHTS"]]+=1
    if r["ORIGIN"] not in aircraftStringCount:
        aircraftStringCount[r["ORIGIN"]] = []
    origin_dict_og.update({i : originAirport(r["ORIGIN"], r["CRS_DEP_DATETIME"], r["DAY_OF_MONTH"])})
    destination_dict_og.update({i : destinationAirport(r["DEST"], r["CRS_ARR_DATETIME"], r["DAY_OF_MONTH"],
                                                       timedelta(days=0,hours=0,minutes=0,seconds=0), 0.0, 0)})
    if r["CRS_DEP_DATETIME"] not in object_dict_dep_sort:
        object_dict_dep_sort[r["CRS_DEP_DATETIME"]] = []
        object_dict_dep_sort[r["CRS_DEP_DATETIME"]].append(FlightLeg(r["ORIGIN"],r["DEST"],r["FLIGHTS"],r["DAY_OF_MONTH"],
                                                                     r["ARR_DELAY_NEW"], r["CRS_DEP_DATETIME"],r["CRS_ARR_DATETIME"]
                                                                     ,0,0.0,None,0.0, 0, None, 0, 0, 0, r["CLUSTER_ID"]
                                                                     , 0, np.inf
                                                                     ,r["Fleet"], [], [], [], []))
    else:
        object_dict_dep_sort[r["CRS_DEP_DATETIME"]].append(FlightLeg(r["ORIGIN"],r["DEST"],r["FLIGHTS"],r["DAY_OF_MONTH"],
                                                                     r["ARR_DELAY_NEW"], r["CRS_DEP_DATETIME"],r["CRS_ARR_DATETIME"]
                                                                     ,0,0.0,None,0.0, 0, None, 0, 0, 0, r["CLUSTER_ID"]
                                                                     , 0, np.inf
                                                                     ,r["Fleet"], [], [], [], []))


sorted_timestamp = sorted(object_dict_dep_sort)
allAirports = list(set(allAirports))
allAircrafts = list(set(allAircrafts))

Algorithm 1 and 2

In [ ]:
first_node_origin = []
first_node_list = []
for i in range(len(object_dict_og)):
    if i in object_dict_copy:
        res = if_first_node(object_dict_copy[i], object_dict_copy)
        if res:
            object_dict_copy[i].sum_priority_idx = object_dict_copy[i].priorityIndex
            object_dict_copy[i].elapsed_time = (object_dict_copy[i].arr_datetime - object_dict_copy[i].dep_datetime).total_seconds()
            first_node_origin.append(object_dict_copy[i].origin)
            first_node_list.append(i)

In [ ]:
#t0 = time.process_time()
#fl_list = []
finalDelay = []
final_string_list = []
final_string_str = []
#f_assigned = []
#Setting delayWeight to reflect their respective inclusion in the objective function. 0 indicates not be included.
delayWeight = 1
#penaltyCost parameter set to be used in the objective function, discussion for the selection of the value can be found in Section 7.2.3 of the paper
penaltyCost = 1000
#gamma1 and gamma2 indicate control parameters in the aggregation term as depicted in equation 11 of the paper
gamma2 = 1
gamma1 = 1
sol = []
#s_delay = []
flightAssigned = []
sol.append(-99)
#tsearch = []
#topt = []
#tpath = []
#tlink = []
#itr = 1
dispPathList = []
while len(sol) != 0:
    #ts1 = time.process_time()
    first_node_tuple = []
    first_node_list = []
    for i in range(len(object_dict_og)):
        if i in object_dict_copy:
            res = if_first_node(object_dict_copy[i], object_dict_copy)
            if res:
                first_node_tuple.append((i, object_dict_copy[i].origin))
                first_node_list.append(i)

    #temp_dummy_list = []
    #dummy_terminal_node_list = []
    #dummy_terminal_node_str_list = []

    #tl1 = time.process_time()
    #for i in range(len(object_dict_og)):

        if i in object_dict_copy:

            if object_dict_copy[i].processed == 0:
                best_prop_delay, best_prev_node_idx = get_best_feasible_link(object_dict_copy[i], object_dict_copy, gamma1, gamma2)
                if best_prev_node_idx != -99 and best_prop_delay != np.inf
                    object_dict_copy[i].pointer = best_prev_node_idx
                    object_dict_copy[i].total_propagated_delay = best_prop_delay
                    object_dict_copy[i].processed = 1


            if object_dict_copy[i].processed == 1:
                if object_dict_copy[i].pointer != 'dummy_start':
                    for f in first_node_tuple:
                        if object_dict_copy[i].pointer == f[0]:
                            object_dict_copy[i].firstnodeorigin = f[1]

                    if object_dict_copy[i].firstnodeorigin == None:
                        object_dict_copy[i].firstnodeorigin = object_dict_copy[object_dict_copy[i].pointer].firstnodeorigin


    #tl2 = time.process_time()
    #tlink.append(tl2 - tl1)

    #all_nodes_idx = object_dict_copy.keys()
    all_pointer_idx = []
    start_pointer_idx = []

    for i in range(len(object_dict_og)):

        if i in object_dict_copy:

            if object_dict_copy[i].processed == 1:
                if object_dict_copy[i].pointer != 'dummy_start':
                    all_pointer_idx.append(object_dict_copy[i].pointer)

                if object_dict_copy[i].pointer == 'dummy_start':
                    start_pointer_idx.append(i)

    for a in all_nodes_idx:

        if a not in all_pointer_idx:
            temp_dummy_list.append(a)
    for d in temp_dummy_list:
        if d not in start_pointer_idx:
            if object_dict_copy[d].processed == 1:
                dummy_terminal_node_list.append(d)
                dummy_terminal_node_str_list.append(object_dict_copy[d].origin+'-'+object_dict_copy[d].dest)


    #tp1 = time.process_time()
    path_list, path_list_str, time_list = path_constructor(object_dict_copy , dummy_terminal_node_list , dummy_terminal_node_str_list)

    #tp2 = time.process_time()
    #tpath.append(tp2 - tp1)

    prop_delay_list = []
    sflights = []
    assignedFlights = []

    for pl in path_list:
        for p in pl:
            if p not in sflights:
                sflights.append(p)
            if p not in assignedFlights:
                if object_dict_copy[p].priorityAssigned == 1:
                    assignedFlights.append(p)


    #for i in range(len(path_list)):
        #prop_delay_list.append(object_dict_copy[path_list[i][len(path_list[i]) - 1]].total_propagated_delay)

    orgCounter = []
    if len(path_list) > 0:
        to1 = time.process_time()
        sol, idx, flt, sdelay = route_optimizer(path_list, prop_delay_list, path_list_str , sflights, assignedFlights , itr, penaltyCost , delayWeight,)

        for i in idx:
            dispPathList.append(path_list[i])
        to2 = time.process_time()
        topt.append(to2-to1)
        itr+=1
        finalDelay.append(sum(sdelay))
        if len(flt) > 0:
            for fll in flt:
                f_assigned.append(fll)

        for s in sdelay:
            s_delay.append(s)

        sol_path = []
        flightsAfterselection = []
        for i in idx:
            sol_path.append(time_list[i])
            for pl in path_list[i]:
                if pl not in flightsAfterselection:
                    flightsAfterselection.append(pl)


        if len(sol) > 0:
            for i in idx:
                # print("Printing path solution: ", path_list[i])
                for j in path_list[i]:
                        if j in first_node_list:
                            orgCounter.append((object_dict_copy[j].origin, object_dict_copy[path_list[i][-1]].dest))

                        del object_dict_copy[j]

    if len(path_list) == 0:
        sol = []

    if len(path_list) > 0:
        for i in idx:
            final_string_list.append(path_list[i])
            final_string_str.append(path_list_str[i])


    for i in range(len(object_dict_og)):
        if i in object_dict_copy:
            object_dict_copy[i].pointer = None
            object_dict_copy[i].processed = 0
            object_dict_copy[i].elapsed_time = 0.0
            object_dict_copy[i].total_propagated_delay = 0.0

    #ts2 = time.process_time()
    #tsearch.append(ts2 - ts1)

#t1 = time.process_time()

#print("Time elapsed: ", t1-t0)

Algorithm 3

In [ ]:
#To verify if the node is first node
def if_first_node(node, node_dict):

    res = False # Set default value of the node as not the first node
    c = 0

    for i in range(len(object_dict_og)):

        if i in node_dict:

            if node.origin == node_dict[i].dest:

                # Check if the departure time of the node is the least among all the nodes
                if node.dep_datetime > node_dict[i].arr_datetime:
                    if node.day >= node_dict[i].day:
        #                 if node.origin == node_dict[i].dest:
                        c += 1
                        break

    if c == 0: # Even if 1 node found satisfying the above conditions

        res = True
        node.processed = 1
        node.pointer = 'dummy_start'

    return res